In [1]:
%cd /content/drive/MyDrive/Deep Learning/TFOD02.x_Setup

/content/drive/MyDrive/Deep Learning/TFOD02.x_Setup


In [2]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

     |████████████████████████████████| 462 kB 3.3 MB/s 
     |████████████████████████████████| 352 kB 4.5 MB/s 


In [3]:
!pip install pycocotools

In [4]:
%%bash 
cd models/research/object_detection/packages/tf2/
pip install .

Processing /content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research/object_detection/packages/tf2
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1206 sha256=ec72492490ef0961533144f937eaeca1ea56e1de8344582371891fafd87a7051
  Stored in directory: /tmp/pip-ephem-wheel-cache-hf4qvx1t/wheels/93/bd/44/a648cee4220823f706f064833f64c689823dd67ec3cc9a51bb
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=6326ac5e702334ba9bd951f5abf76a465879711cd1478a36fc622e65e5bad03e
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=324f1d1f651a61b739ff697f5dde36801245a5f34be8382764dac9cb13bbd218
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.wh

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Prediction

In [11]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [13]:
%cd /content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research

/content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research


In [14]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [15]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [16]:
#custom model loading
import pathlib

def load_custom_model(model_name):
  model_file = model_name

  model_dir = pathlib.Path(model_file)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [17]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'Chess_Piece_Detector/training/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [18]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('Chess_Piece_Detector/chess_dataset/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('Chess_Piece_Detector/chess_dataset/test/0b47311f426ff926578c9d738d683e76_jpg.rf.f45eee1c0901a80069f6bbff97523c75.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/1c0060ef868bdc326ce5e6389cb6732f_jpg.rf.0d044deaff923c0579c9e00c7d02f110.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/2f6fb003bb89cd401322a535acb42f65_jpg.rf.91c5a8446c434264d9c8f7b04269f329.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/410993714e325a1de3e394ffe860df3a_jpg.rf.2bad072566b5d1f71e1ee141121e3ed2.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/4e3117459d759798537eb52cf5bf534d_jpg.rf.4f037ca3aeaa0936f6dbf20a1a273e00.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/5a35ba2ec3e0d0b2b12b1758a8ac29aa_jpg.rf.9ca03b5207b737ee605805d88a0619b2.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/654bb8835258b26c466b1c19893df451_jpg.rf.598adc2540fbdee36d8f704d3d7b34df.jpg'),
 PosixPath('Chess_Piece_Detector/chess_dataset/test/685b860d412b91f5d4f7f9e6

In [19]:
model_name = '/content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research/Chess_Piece_Detector/chess_detector'
detection_model = load_custom_model(model_name)

In [20]:
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'input_tensor:0' shape=(1, None, None, 3) dtype=uint8>, <tf.Tensor 'unknown:0' shape=() dtype=resource>, <tf.Tensor 'unknown_0:0' shape=() dtype=resource>, <tf.Tensor 'unknown_1:0' shape=() dtype=resource>, <tf.Tensor 'unknown_2:0' shape=() dtype=resource>, <tf.Tensor 'unknown_3:0' shape=() dtype=resource>, <tf.Tensor 'unknown_4:0' shape=() dtype=resource>, <tf.Tensor 'unknown_5:0' shape=() dtype=resource>, <tf.Tensor 'unknown_6:0' shape=() dtype=resource>, <tf.Tensor 'unknown_7:0' shape=() dtype=resource>, <tf.Tensor 'unknown_8:0' shape=() dtype=resource>, <tf.Tensor 'unknown_9:0' shape=() dtype=resource>, <tf.Tensor 'unknown_10:0' shape=() dtype=resource>, <tf.Tensor 'unknown_11:0' shape=() dtype=resource>, <tf.Tensor 'unknown_12:0' shape=() dtype=resource>, <tf.Tensor 'unknown_13:0' shape=() dtype=resource>, <tf.Tensor 'unknown_14:0' shape=() dtype=resource>, <tf.Tensor 'unknown_15:0' shape=() dtype=resource>, <tf.Tensor 'unknown_16:0' shape=() dtype=resource>, <tf.Tenso

In [21]:
detection_model.signatures['serving_default'].output_dtypes

{'detection_anchor_indices': tf.float32,
 'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32,
 'raw_detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32}

In [22]:
detection_model.signatures['serving_default'].output_shapes

{'detection_anchor_indices': TensorShape([1, 100]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'detection_classes': TensorShape([1, 100]),
 'detection_multiclass_scores': TensorShape([1, 100, 7]),
 'detection_scores': TensorShape([1, 100]),
 'num_detections': TensorShape([1]),
 'raw_detection_boxes': TensorShape([1, 51150, 4]),
 'raw_detection_scores': TensorShape([1, 51150, 7])}

In [23]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [24]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [25]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

Output hidden; open in https://colab.research.google.com to view.

# Custom Model Preparation

In [5]:
%cd models/research/Chess_Piece_Detector/

/content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research/Chess_Piece_Detector


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 retinanet-50

## Generating tfRecords from csv

In [ ]:
!python generate_tfrecord.py --csv_input=Chess_Piece_Detector/chess_dataset/train/train.csv --image_dir=Chess_Piece_Detector/chess_dataset/train --output_path=Chess_Piece_Detector/train.record

In [ ]:
!python generate_tfrecord.py --csv_input=Chess_Piece_Detector/chess_dataset/valid/test.csv --image_dir=Chess_Piece_Detector/chess_dataset/valid --output_path=Chess_Piece_Detector/val.record

# Training

In [9]:
cd ..

/content/drive/MyDrive/Deep Learning/TFOD02.x_Setup/models/research


In [ ]:
!python model_main_tf2.py --model_dir=training --pipeline_config_path=Chess_Piece_Detector/training/pipeline.config

In [10]:
!python exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=Chess_Piece_Detector/training/pipeline.config --trained_checkpoint_dir=Chess_Piece_Detector/training --output_directory=Chess_Piece_Detector/chess_detector 

2022-03-04 15:50:32.010672: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0304 15:50:36.747794 140074786125696 deprecation.py:615] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
2022-03-04 15:51:18.612147: W tensorflow/python/util/util.cc:368] Sets are not currently 